In [1]:
# import the necessary packages
import numpy as np
import imutils
import cv2
import os
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from imutils import paths

In [2]:
#Functions
def image_to_feature_vector(image, size=(32, 32)):
	# resize the image to a fixed size, then flatten the image into
	# a list of raw pixel intensities
	return cv2.resize(image, size).flatten()

def extract_color_histogram(image, bins=(8, 8, 8)):
	# extract a 3D color histogram from the HSV color space using
	# the supplied number of `bins` per channel
	hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
	hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
		[0, 180, 0, 256, 0, 256])
    	# handle normalizing the histogram if we are using OpenCV 2.4.X
	if imutils.is_cv2():
		hist = cv2.normalize(hist)

	# otherwise, perform "in place" normalization in OpenCV 3
	else:
		cv2.normalize(hist, hist)

	# return the flattened histogram as the feature vector
	return hist.flatten()

In [3]:
#Images
# grab the list of images that we'll be describing
print("[INFO] describing images...")
imagePaths = list(paths.list_images("./train_vega"))
print imagePaths
# initialize the raw pixel intensities matrix, the features matrix,
# and labels list
rawImages = []
features = []
labels = []

[INFO] describing images...
['./train_vega/skara.12.jpg', './train_vega/skara.11.jpg', './train_vega/skara.14.jpg', './train_vega/hibou.5.jpg', './train_vega/hibou.6.jpg', './train_vega/skara.18.jpg', './train_vega/hibou.9.jpg', './train_vega/hibou.1.jpg', './train_vega/hibou.11.jpg', './train_vega/skara.09.jpg', './train_vega/skara.07.jpg', './train_vega/skara.13.jpg', './train_vega/hibou.16.jpg', './train_vega/hibou.14.jpg', './train_vega/skara.19.jpg', './train_vega/hibou.3.jpg', './train_vega/skara.04.jpg', './train_vega/skara.03.jpg', './train_vega/skara.01.jpg', './train_vega/skara.16.jpg', './train_vega/skara.20.jpg', './train_vega/hibou.4.jpg', './train_vega/skara.05.jpg', './train_vega/hibou.10.jpg', './train_vega/hibou.8.jpg', './train_vega/skara.06.jpg', './train_vega/hibou.2.jpg', './train_vega/skara.17.jpg', './train_vega/hibou.15.jpg', './train_vega/skara.02.jpg', './train_vega/skara.15.jpg', './train_vega/skara.10.jpg', './train_vega/skara.08.jpg', './train_vega/hibou.7.

In [4]:
# loop over the input images
for (i, imagePath) in enumerate(imagePaths):
	# load the image and extract the class label (assuming that our
	# path as the format: /path/to/dataset/{class}.{image_num}.jpg
	image = cv2.imread(imagePath)
	image_flip = cv2.flip(image,1)
	label = imagePath.split(os.path.sep)[-1].split(".")[0]
    
	# extract raw pixel intensity "features", followed by a color
	# histogram to characterize the color distribution of the pixels
	# in the image
	pixels = image_to_feature_vector(image)
	pixels_flip = image_to_feature_vector(image_flip)
	hist = extract_color_histogram(image)
	hist_flip = extract_color_histogram(image_flip)

	# update the raw images, features, and labels matricies,
	# respectively
	rawImages.append(pixels)
	rawImages.append(pixels_flip)
	features.append(hist)
	features.append(hist_flip)
	labels.append(label)
	labels.append(label)

	# show an update every 1,000 images
	if i > 0 and i % 1000 == 0:
		print("[INFO] processed {}/{}".format(i, len(imagePaths)))

In [5]:
rawImages = np.array(rawImages)
features = np.array(features)
labels = np.array(labels)
# partition the data into training and testing splits, using 75%
# of the data for training and the remaining 25% for testing
(trainRI, testRI, trainRL, testRL) = train_test_split(
	rawImages, labels, test_size=0.25, random_state=42)
(trainFeat, testFeat, trainLabels, testLabels) = train_test_split(
	features, labels, test_size=0.25, random_state=42)

In [18]:
# train and evaluate a SVM classifer on the raw pixel intensities
print("[INFO] evaluating raw pixel accuracy...")
model =  SVC()
model.fit(trainRI, trainRL)
acc = model.score(testRI, testRL)
print("[INFO] raw pixel accuracy: {:.2f}%".format(acc * 100))

[INFO] evaluating raw pixel accuracy...
[INFO] raw pixel accuracy: 66.67%


In [19]:
# train and evaluate a SVM classifer on the histogram
# representations
print("[INFO] evaluating histogram accuracy...")
model =  SVC()
model.fit(trainFeat, trainLabels)
acc = model.score(testFeat, testLabels)
print("[INFO] histogram accuracy: {:.2f}%".format(acc * 100))

[INFO] evaluating histogram accuracy...
[INFO] histogram accuracy: 66.67%
